## Getting close price from CryptoCompare

In [2]:
import pandas as pd
import numpy as np
import json
import requests

In [3]:
# Function to load JSON
def loadJSON(filepath):
    with open(filepath) as file:
        return json.load(file)

In [4]:
# Loading JSON
data = loadJSON('../../data/json/main-tokens-clean.json')
data

[{'gecko_id': 'harmony',
  'tvl': 1895183.4834775673,
  'tokenSymbol': 'ONE',
  'cmcId': '3945',
  'name': 'Harmony',
  'chainId': 1666600000},
 {'gecko_id': 'mantle',
  'tvl': 461335459.59074324,
  'tokenSymbol': 'MNT',
  'cmcId': '27075',
  'name': 'Mantle',
  'chainId': 5000},
 {'gecko_id': 'aurora-near',
  'tvl': 16711904.96719923,
  'tokenSymbol': 'AURORA',
  'cmcId': '14803',
  'name': 'Aurora',
  'chainId': 1313161554},
 {'gecko_id': 'moonbeam',
  'tvl': 25572759.005970616,
  'tokenSymbol': 'GLMR',
  'cmcId': '6836',
  'name': 'Moonbeam',
  'chainId': 1284},
 {'gecko_id': 'kava',
  'tvl': 148889498.48075423,
  'tokenSymbol': 'KAVA',
  'cmcId': '4846',
  'name': 'Kava',
  'chainId': None},
 {'gecko_id': 'gnosis',
  'tvl': 251643065.26045218,
  'tokenSymbol': 'GNO',
  'cmcId': '1659',
  'name': 'Gnosis',
  'chainId': 100},
 {'gecko_id': 'optimism',
  'tvl': 638548026.7500983,
  'tokenSymbol': 'OP',
  'cmcId': '11840',
  'name': 'Optimism',
  'chainId': 10},
 {'gecko_id': 'fantom',

## Defining endpoint properties

In [5]:
# (There's no need for API key)

# Endpoint URL
url = f'https://min-api.cryptocompare.com/data/v2/histoday?fsym='

# Request header
headers = {
    'Accepts': 'application/json'
}

In [8]:
# Inicializar listas para nombres de tokens
successful_tokens = []
failed_tokens = []

# Counter to check the amount of tokens processed
counter = 0
for i, token in enumerate(data):
    # Construct the URL
    url_to_call = f"{url}{token['tokenSymbol']}&tsym=USD&limit=5"
    print(f'Calling URL: {url_to_call}')
    
    response = requests.get(url_to_call, headers=headers)

    # Checking if it's successful
    if response.status_code == 200:
        response_data = response.json()

        # Imprimir la respuesta para depurar
        print(f'Response for {token["name"]}:', response_data)

        # Verificar si 'Data' contiene datos
        if 'Data' in response_data:
            # Verificar si 'Data' es un diccionario que contiene la lista
            if isinstance(response_data['Data'], dict) and 'Data' in response_data['Data']:
                ohlc_data = [
                    {'timestamp': entry['time'], 'close': entry['close']}
                    for entry in response_data['Data']['Data']
                ]

                # Añadir el OHLC específico para este token
                token['OHLC'] = ohlc_data
                successful_tokens.append(token['name'])  # Agregar a la lista de exitosos
                counter += 1
                
                print(f'{token["name"]}: Successful')
            else:
                print(f'No OHLC data available for {token["name"]}. Data: {response_data["Data"]}')
                failed_tokens.append(token['name'])  # Agregar a la lista de fallidos
        else:
            print(f'No data key found in response for {token["name"]}')
            failed_tokens.append(token['name'])  # Agregar a la lista de fallidos
    else:
        print(f'Error for {token["name"]}: {response_data.get("Message", "Unknown error")}')
        failed_tokens.append(token['name'])  # Agregar a la lista de fallidos

print('Successful requests:', counter)
print('Successful tokens:', successful_tokens)
print('Failed tokens:', failed_tokens)


Calling URL: https://min-api.cryptocompare.com/data/v2/histoday?fsym=ONE&tsym=USD&limit=5
Response for Harmony: {'Response': 'Success', 'Message': '', 'HasWarning': False, 'Type': 100, 'RateLimit': {}, 'Data': {'Aggregated': False, 'TimeFrom': 1726531200, 'TimeTo': 1726963200, 'Data': [{'time': 1726531200, 'high': 0.01199, 'low': 0.01128, 'open': 0.01142, 'volumefrom': 968820, 'volumeto': 11361.07, 'close': 0.01197, 'conversionType': 'direct', 'conversionSymbol': ''}, {'time': 1726617600, 'high': 0.01245, 'low': 0.01134, 'open': 0.01197, 'volumefrom': 973020, 'volumeto': 11469.37, 'close': 0.01242, 'conversionType': 'direct', 'conversionSymbol': ''}, {'time': 1726704000, 'high': 0.01318, 'low': 0.01242, 'open': 0.01242, 'volumefrom': 1005530, 'volumeto': 12998.23, 'close': 0.01282, 'conversionType': 'direct', 'conversionSymbol': ''}, {'time': 1726790400, 'high': 0.01359, 'low': 0.01265, 'open': 0.01282, 'volumefrom': 800500, 'volumeto': 10674.38, 'close': 0.01346, 'conversionType': 'di

In [7]:
with open('../../data/json/tokens-tvl_prueba.json', 'w') as outfile:
    json.dump(data, outfile, indent=4)
    print("JSON successfully saved at '../../data/json/tokens-tvl_prueba.json'")

JSON successfully saved at '../../data/json/tokens-tvl_prueba.json'
